In [8]:
from wlcorr import EncoderDecoderStaticDataset, EncoderDecoder1DCNN
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

In [3]:
dataset = EncoderDecoderStaticDataset('/home/shivam/DKLabs/OilGasProject/WellLogCorrelation/data')

In [4]:
dataset.__len__()

98

In [5]:
dataloader = DataLoader(dataset, batch_size = 10, shuffle = True,)

In [6]:
model = EncoderDecoder1DCNN()
model.to('cuda')

EncoderDecoder1DCNN(
  (encoder): Sequential(
    (0): Conv1d(2, 5, kernel_size=(10,), stride=(1,), padding=(9,))
    (1): SELU()
    (2): Conv1d(5, 10, kernel_size=(10,), stride=(1,), padding=(9,))
    (3): SELU()
    (4): Conv1d(10, 10, kernel_size=(10,), stride=(1,), padding=(9,))
    (5): SELU()
    (6): Conv1d(10, 10, kernel_size=(10,), stride=(1,), padding=(9,))
    (7): SELU()
    (8): Conv1d(10, 1, kernel_size=(10,), stride=(1,), padding=(9,))
    (9): SELU()
    (10): Linear(in_features=145, out_features=10, bias=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(1, 5, kernel_size=(10,), stride=(1,))
    (1): SELU()
    (2): ConvTranspose1d(5, 10, kernel_size=(10,), stride=(1,))
    (3): SELU()
    (4): ConvTranspose1d(10, 10, kernel_size=(10,), stride=(1,))
    (5): SELU()
    (6): ConvTranspose1d(10, 5, kernel_size=(10,), stride=(1,))
    (7): SELU()
    (8): ConvTranspose1d(5, 5, kernel_size=(10,), stride=(1,))
    (9): SELU()
    (10): ConvTranspose1d(5, 2, kernel

In [7]:
model.eval()

EncoderDecoder1DCNN(
  (encoder): Sequential(
    (0): Conv1d(2, 5, kernel_size=(10,), stride=(1,), padding=(9,))
    (1): SELU()
    (2): Conv1d(5, 10, kernel_size=(10,), stride=(1,), padding=(9,))
    (3): SELU()
    (4): Conv1d(10, 10, kernel_size=(10,), stride=(1,), padding=(9,))
    (5): SELU()
    (6): Conv1d(10, 10, kernel_size=(10,), stride=(1,), padding=(9,))
    (7): SELU()
    (8): Conv1d(10, 1, kernel_size=(10,), stride=(1,), padding=(9,))
    (9): SELU()
    (10): Linear(in_features=145, out_features=10, bias=True)
  )
  (decoder): Sequential(
    (0): ConvTranspose1d(1, 5, kernel_size=(10,), stride=(1,))
    (1): SELU()
    (2): ConvTranspose1d(5, 10, kernel_size=(10,), stride=(1,))
    (3): SELU()
    (4): ConvTranspose1d(10, 10, kernel_size=(10,), stride=(1,))
    (5): SELU()
    (6): ConvTranspose1d(10, 5, kernel_size=(10,), stride=(1,))
    (7): SELU()
    (8): ConvTranspose1d(5, 5, kernel_size=(10,), stride=(1,))
    (9): SELU()
    (10): ConvTranspose1d(5, 2, kernel

In [7]:
data = iter(dataloader).next()
data = data.to('cuda')

In [8]:
ret = model(data)

In [44]:
def loss(output:torch.Tensor, target:torch.Tensor):
    b, n, _ = output.shape
    output = output.div(output.square().sum(dim=2).view(b, n, 1))
    target = target.div(target.square().sum(dim=2).view(b, n, 1))
    
    l = output.multiply(target).sum(dim=2).exp().mean(dim = 1).mean()
    return l

In [10]:
t1 = torch.zeros(10, 100)
t2 = torch.ones(10, 100)
t3 = torch.rand(10, 100)

In [11]:
new_t = torch.cat((t1, t2, t3), dim = 0)

In [12]:
final_tensor = torch.permute((new_t.view(3, 10, -1)), (1, 2, 0))

In [13]:
torch.tensordot(final_tensor, final_tensor, dims = ([1, 2], [1, 2])).shape

torch.Size([10, 10])

In [31]:
norm = final_tensor.square().sum(dim=2).view(10, 100, 1)
print(norm.shape, final_tensor.shape)
final_tensor.div(norm).shape

torch.Size([10, 100, 1]) torch.Size([10, 100, 3])


torch.Size([10, 100, 3])

In [27]:
final_tensor.square().sum(dim=2).view(10, 1, 100).expand(-1, 3, -1).shape

torch.Size([10, 3, 100])

In [14]:
torch.sum(torch.multiply(final_tensor, final_tensor), dim = 2).shape

torch.Size([10, 100])

In [47]:
torch.norm(final_tensor, dim=2).shape

torch.Size([10, 100])

In [41]:
final_tensor.multiply(final_tensor).sum(dim=2).exp().mean(dim = 1).mean()

tensor(3.9833)

In [9]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output.shape

torch.Size([2, 3])

In [15]:
loss = nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)

In [16]:
output

tensor(1.7330, grad_fn=<MseLossBackward0>)

In [45]:
loss(final_tensor, final_tensor)

tensor(2.2205)

In [62]:
input1 = torch.randn(10, 100, 3) #torch.tensor([[2, 3, 4]], dtype=torch.float32)
input2 = torch.randn(10, 100, 3) #torch.tensor([[-2, -3, -4]], dtype=torch.float32)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)

In [64]:
output.shape

torch.Size([10, 3])